<h1 style="text-align: center;">AAG - Méthodes Classiques en IA</h1>


## ZeroR / OneRule

**<ins>Dataset R6 :</ins>**
<style>
    .heatMap {
        width: 70%;
        text-align: center;
    }
    .heatMap th {
        background: grey;
        word-wrap: break-word;
        text-align: center;
    }

</style>

<ins>Types de features</ins>
<div class="heatMap">
									

|   Nom des features           |  Type       |
|:---------------:|:-----------:|
|   skillrank  |   Nom (string)   |
|   operator  |   Nom (string)  |
|   primaryweapon   |   Nom (string)   |
|   secondaryweapon   |   nom (string)   |
|   secondarygadget   |   nom (string)   |
|   k/d   |   numeric (int)   |
|   target   |   numeric (int)   |
</div>

<ins>Séparation du corpus</ins>
<div class="heatMap">

|                 | Train (2/3) | Test(1/3) | Total / type de w/r |
|:---------------:|:-----:|:----:|:------:|
| 1 (w/r positif)  |   398  |  199  |   597   |
| 0 (w/r negatif) |   1035  |  518  |   1553   |
| Total / corpus |   1433 | 717  |  2150   |

</div>

### Préparation du dataset & création des corpus

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Chargement du dataset dans une DataFrame
dataset_R6 = pd.read_csv("./DatasetR6/S5_operators.csv", sep=';')

# on calcule le w/r
dataset_R6.insert(dataset_R6.shape[1],"w/r", (dataset_R6["nbwins"]/dataset_R6["nbpicks"]), True) 
dataset_R6.drop(columns=["nbwins","nbpicks","role"], inplace=True)

# on calcule le k/d
dataset_R6.loc[dataset_R6['nbdeaths'] == 0] = 1
# on a une ligne qui fait que des 1 partout 
dataset_R6.drop(axis=0, index=dataset_R6[dataset_R6['operator'] == 1].index, inplace=True)
dataset_R6['k/d'] = dataset_R6['nbkills'] / dataset_R6['nbdeaths']


# on merge et enleve les colonnes plateformes et dateid sans perdre les données associées
dataset_R6 = dataset_R6.groupby(['skillrank','operator', 'primaryweapon', 'secondaryweapon', 'secondarygadget'], as_index=False)[['w/r', 'k/d']].mean()

# discretisation de w/r
dataset_R6.insert(dataset_R6.shape[1],"target", (dataset_R6["w/r"]>=0.5).astype(int), True) 

dataset_R6.drop(columns=["w/r"], inplace=True)

#discrétisation de k/d
dataset_R6['k/d'] = (dataset_R6["k/d"]>=1.0).astype(int)


# changement de la taille d'affichage des dataframes
#pd.set_option('display.max_rows', dataset_R6.shape[0]+1)

corpus_annot = dataset_R6.copy()
corpus_annot = corpus_annot[['target']]

corpus_non_annot = dataset_R6.copy()
corpus_non_annot.drop(columns=["target"], inplace=True)

x_train, x_test, y_train, y_test = train_test_split(corpus_non_annot, corpus_annot, test_size=1/3, random_state=42, stratify=corpus_annot)

corpus_test_non_annot = x_test.copy()

corpus_train = x_train.copy()
corpus_train['target'] = y_train.copy()

corpus_verif = x_test.copy()
corpus_verif['target'] = y_test.copy()

dataset_R6

#dataset_R6.groupby(['target']).count()

,skillrank,operator,primaryweapon,secondaryweapon,secondarygadget,k/d,target
0,Bronze,BOPE-CAPITAO,M249,PRB92,CLAYMORE,0,0
1,Bronze,BOPE-CAPITAO,M249,PRB92,STUN GRENADE,0,0
2,Bronze,BOPE-CAPITAO,PARA-308,PRB92,CLAYMORE,0,0
3,Bronze,BOPE-CAPITAO,PARA-308,PRB92,STUN GRENADE,0,1
4,Bronze,BOPE-CAVEIRA,M12,PRB92,BARBED WIRE,0,1
...,...,...,...,...,...,...,...
2145,Unranked,SWAT-THERMITE,556XI,M45 MEUSOC,SMOKE GRENADE,0,0
2146,Unranked,SWAT-THERMITE,M1014,5.7 USG,CLAYMORE,0,0
2147,Unranked,SWAT-THERMITE,M1014,5.7 USG,SMOKE GRENADE,0,0
2148,Unranked,SWAT-THERMITE,M1014,M45 MEUSOC,CLAYMORE,0,0


### Application du ZeroR :

In [2]:
def classe_plus_commune(corp_annote):
    # creation d'un tableau de taille du nombre de classe
    a = [0] * corp_annote.target.nunique()
    # on compte le nombre d'occurence de chaque classe
    for x in corp_annote.itertuples():
        a[x.target] += 1
    return a.index(max(a))


def zeror_hyp(corpus_train, corp_non_annote):
    classe_moyenne = classe_plus_commune(corpus_train)
    corp_non_annote['target'] = classe_moyenne
    return corp_non_annote

#pd.set_option('display.max_rows', None)


def tester_accuracy(corpus_train,corpus_non_annote):
    # on fait l'hypothese et on compare avec le corpus de verification
    nb_true, nb_false  = zeror_hyp(corpus_train, corpus_non_annote).eq(corpus_verif)['target'].value_counts()
    print(f' nous avons {nb_true} bonnes annotations et {nb_false} mauvaises annotations sur {nb_true+nb_false} données ce qui donne un accuracy de {(nb_true/(nb_true+nb_false))*100}%')

tester_accuracy(corpus_train, corpus_test_non_annot)

 nous avons 518 bonnes annotations et 199 mauvaises annotations sur 717 données ce qui donne un accuracy de 72.24546722454673%


#### Accuracy de 72.24546722454673% <-- notre baseline

### Application du OneRule :

In [3]:
# Chargement du dataset dans une DataFrame
corpus_annot = dataset_R6.copy()
corpus_annot = corpus_annot[['target']]

corpus_non_annot = dataset_R6.copy()
corpus_non_annot.drop(columns=["target"], inplace=True)

x_train, x_test, y_train, y_test = train_test_split(corpus_non_annot, corpus_annot, test_size=1/3, random_state=42, stratify=corpus_annot)

corpus_test_non_annot = x_test.copy()

corpus_train = x_train.copy()
corpus_train['target'] = y_train.copy()

corpus_verif = x_test.copy()
corpus_verif['target'] = y_test.copy()

In [15]:
'''
Trouve le descripteur qui minimise le taux d'erreur et annote le corpus passé en paramètre
'''

def oneR_hyp(corp_train, corp_non_annote):

    meilleur_attribut = ''
    meilleur_attribut_cer = 1
    dic_att = {}
    
    if('target' in corp_non_annote.columns):
        corp_non_annote.drop(columns=["target"], inplace=True)
    
    # Recherche du descripteur qui minimise le taux d'erreur
    for nom_attribut in corp_non_annote.columns:
        attribut = corp_non_annote[nom_attribut]
        tab_val = []
        for valeur in attribut.unique():
            #print(f"pour la valeur {valeur} la target max est \n {corp_train.loc[corp_train[nom_attribut] == valeur]['target'].value_counts().idxmax()}")
            dic_att[valeur] = corp_train.loc[corp_train[nom_attribut] == valeur]['target'].value_counts().idxmax()
        target_hyp = corp_non_annote[nom_attribut].map(dic_att)
        
        salut = corp_non_annote.copy()
        salut['target'] = target_hyp
        cer = calcul_cer(salut, salut['target'].eq(corpus_verif['target']).value_counts(), nom_attribut)
        print(f" l'attribut {nom_attribut} a un CER de {cer}")
        if(cer < meilleur_attribut_cer):
            meilleur_attribut = nom_attribut
            meilleur_attribut_cer = cer
    
    print(f"\nL'attribut avec le CER le plus petit est {nom_attribut} avec un CER de {cer}\n")
    corp_non_annote['target'] = corp_non_annote[meilleur_attribut]
    #print(dic_att)
    return corp_non_annote


'''
Calcul le taux d'erreur de la classification

Rappel :
Cer = (# instances mal classées) / (# instances classées)
'''
def calcul_cer(corp_verif, tab_vrai_faux, nom_descripteur):
    # on fait l'hypothese et on compare avec le corpus de verification
    if(tab_vrai_faux.size > 1):
        nb_true = tab_vrai_faux[True]
        nb_false = tab_vrai_faux[False]
    else:
        nb_true = tab_vrai_faux[0]
        nb_false = 0
    #print(f' pour le descripteur {nom_descripteur}, nous avons {nb_true} bonnes annotations et {nb_false} mauvaises annotations sur {nb_true+nb_false} données ce qui donne un taux d\'erreur de {(nb_false/(nb_true+nb_false))*100}%')
    return nb_false/(nb_true+nb_false)

#oneR_hyp(corpus_train, x_test)

def tester_accuracy(corpus_train,corpus_non_annote):
    # on fait l'hypothese et on compare avec le corpus de verification
    nb_true, nb_false  = oneR_hyp(corpus_train, corpus_non_annote).eq(corpus_verif)['target'].value_counts()
    print(f' nous avons {nb_true} bonnes annotations et {nb_false} mauvaises annotations sur {nb_true+nb_false} données ce qui donne un accuracy de {(nb_true/(nb_true+nb_false))*100}%')    
    
tester_accuracy(corpus_train, x_test)

 l'attribut skillrank a un CER de 0.2775453277545328
 l'attribut operator a un CER de 0.2412831241283124
 l'attribut primaryweapon a un CER de 0.2482566248256625
 l'attribut secondaryweapon a un CER de 0.28730822873082285
 l'attribut secondarygadget a un CER de 0.28312412831241285
 l'attribut k/d a un CER de 0.16596931659693165

L'attribut avec le CER le plus petit est k/d avec un CER de 0.16596931659693165

 nous avons 598 bonnes annotations et 119 mauvaises annotations sur 717 données ce qui donne un accuracy de 83.40306834030683%


<style>
    .heatMap {
        width: 70%;
        text-align: center;
    }
    .heatMap th {
        background: grey;
        word-wrap: break-word;
        text-align: center;
    }

</style>
Voici les résultat avec le R6

<ins>Taux d'erreur par feature choisie </ins>
<div class="heatMap">


|   Nom des features |     Cer           | 
|:---------------:|:--------------------:|
|   k/d    |         0.165           |
|   operator    |          0.241          |
|   primaryweapon   |        0.248         |
|   skillrank  |         0.278        | 
|   secondarygadget     |         0.283           |
|   secondaryweapon    |         0.287           |


</div>